In [ ]:
# Step 1: Install required libraries
!pip install transformers datasets peft accelerate trl huggingface_hub



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Step 2: Authenticate with Hugging Face
from huggingface_hub import login
login()  # Benard's HF token


In [ ]:
# Step 3: Load and format MentalChat16K dataset
from datasets import load_dataset

dataset = load_dataset("ShenLab/MentalChat16K", split="train[:10]")  # Reduced to 10 samples to save memory


def format_example(example):
    return {
        "text": f"### Instruction:\n{example['input']}\n\n### Response:\n{example['output']}"
    }

dataset = dataset.map(format_example)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.58k [00:00<?, ?B/s]

Interview_Data_6K.csv:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

Synthetic_Data_10K.csv:   0%|          | 0.00/32.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16084 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
# Step 4: Load tokenizer and base model with LoRA config
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType
import torch
model_name = "mistralai/Mistral-7B-v0.1"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, token=True)  # Use token='your_token' if needed
tokenizer.pad_token = tokenizer.eos_token

# Load model with correct dtype and device
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"  # Ensures optimal placement for A100 GPU
)

# Apply LoRA (Low-Rank Adaptation)
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=4,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none"
)

model = get_peft_model(base_model, peft_config)
model.print_trainable_parameters()


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

trainable params: 1,703,936 || all params: 7,243,436,032 || trainable%: 0.0235


In [ ]:
# Step 5: Tokenize dataset
def tokenize_function(example):
    result = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = dataset.map(tokenize_function)




Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
# Step 6: Set training arguments
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,  # A100 can go higher e.g. 2 or 4
    num_train_epochs=1,
    fp16=True,  # Enable mixed precision for faster training
    logging_steps=1,
    save_steps=5,
    save_total_limit=2
)


In [ ]:
# Step 7: Initialize Trainer and fine-tune
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: briankimutai3889 (benardkirwa) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,1.201900
2,1.375200
3,1.612100
4,1.593000
5,1.541500
6,1.623700
7,1.045800
8,1.680300
9,1.407400
10,1.494700


TrainOutput(global_step=10, training_loss=1.4575570225715637, metrics={'train_runtime': 141.7704, 'train_samples_per_second': 0.071, 'train_steps_per_second': 0.071, 'total_flos': 54622955765760.0, 'train_loss': 1.4575570225715637, 'epoch': 1.0})

In [ ]:
# Step 8: Push model and tokenizer to Hugging Face Hub
from huggingface_hub import HfApi, HfFolder
from transformers import AutoTokenizer

# Step 1: Save model locally
model.save_pretrained("finetunedmistral7b")
tokenizer.save_pretrained("finetunedmistral7b")

# Step 2: Login to Hugging Face using your token
HfFolder.save_token()

# Step 3: Push to your repository
model.push_to_hub("Benardkirwa/finetunedmistral7b", use_auth_token=True)
tokenizer.push_to_hub("Benardkirwa/finetunedmistral7b", use_auth_token=True)


/usr/local/lib/python3.11/dist-packages/transformers/utils/hub.py:920: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/6.83M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Benardkirwa/finetunedmistral7b/commit/19b2ebd57b086f62eea6f5d2a20802e0b405ca82', commit_message='Upload tokenizer', commit_description='', oid='19b2ebd57b086f62eea6f5d2a20802e0b405ca82', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Benardkirwa/finetunedmistral7b', endpoint='https://huggingface.co', repo_type='model', repo_id='Benardkirwa/finetunedmistral7b'), pr_revision=None, pr_num=None)